In [1]:
from agents.agent import run_agent

import warnings
from dotenv import load_dotenv

load_dotenv("plash.env")
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

Things to remember to talk about
- prompt caching
- function calling
- reasonign models - thought tokens - need to be passed back - thinking traces need to be passed back
- LLM agent abstractions - less hackable?
- litellm
- advanced shit - https://docs.litellm.ai/blog/anthropic_advanced_features#tool-search
- https://www.anthropic.com/engineering/advanced-tool-use
- dont want a lot of tools
- progressive disclosure - tool search 
- https://docs.litellm.ai/blog/gemini_3_flash
- https://docs.litellm.ai/blog/anthropic_advanced_features
- https://github.com/openai/openai-agents-python
- https://www.anthropic.com/engineering/multi-agent-research-system
- context - tools blow it up
- they all have diff syntax

In [2]:
for rec in run_agent(
    [
        {
            "role": "user",
            "content": "Get the weather fo 4 cities of your choice. If all are the same then do a fifth. Then get the average using the coding tool.",
        }
    ]
):
    print(rec)

{"content":"I'll get the weather for 4 cities at once.","role":"assistant","tool_calls":[{"index":2,"function":{"arguments":"{\"city\": \"New York\"}","name":"get_weather"},"id":"toolu_012nCmyibRNjbGKYbLgwdyCg","type":"function"},{"index":3,"function":{"arguments":"{\"city\": \"London\"}","name":"get_weather"},"id":"toolu_01EpwoU5B9UuG6wvT3m8RFNv","type":"function"},{"index":4,"function":{"arguments":"{\"city\": \"Tokyo\"}","name":"get_weather"},"id":"toolu_01JZ4wAVvf2AJdzBjteKQ2o5","type":"function"},{"index":5,"function":{"arguments":"{\"city\": \"Sydney\"}","name":"get_weather"},"id":"toolu_01GGUDLfGQ39RM7cWscDxSa6","type":"function"}],"function_call":null,"reasoning_content":"The user wants me to get weather for 4 cities, then calculate the average using code. Let me pick 4 cities and get their weather in parallel.","thinking_blocks":[{"type":"thinking","thinking":"The user wants me to get weather for 4 cities, then calculate the average using code. Let me pick 4 cities and get the

In [ ]:
from litellm import completion
from dotenv import load_dotenv

load_dotenv()

response = completion(
    model="gemini/gemini-3-flash-preview",
    messages=[{"role": "user", "content": "Hello!"}],
    reasoning_effort="low",
)

print(response.choices[0].message.content)

In [ ]:
import litellm

litellm.supports_function_calling(model="claude-opus-4-5-20251101")

In [ ]:
from agents.tools import TOOLS, TOOL_FUNCTIONS
from pprint import pprint
import json


def run_agent():
    # Step 1: Initial request
    messages = [
        {
            "role": "user",
            "content": "What is the capital of France? What is 4847847*2409224 What is the weather in Tokyo? Use python to calc todays date. Then take that date and plug back into python and take sqrt.",
        }
    ]

    while True:
        pprint(messages)
        # Call the LLM
        response = litellm.completion(
            model="gpt-5.2",  # "gemini/gemini-3-flash-preview", #claude-opus-4-5-20251101
            messages=messages,
            tools=TOOLS,
            reasoning_effort="low",
        )

        # Append assistant message (thought signatures automatically preserved)
        message = response.choices[0].message

        # If no tool calls, we're done - return the response
        if not message.tool_calls:
            return message.content

        # Otherwise, process tool calls
        messages.append(message)  # Add assistant message with tool calls

        # Execute tool and append result
        for tool_call in message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"Calling tool: {name}({args})")

            # Execute the tool
            result = TOOL_FUNCTIONS[name](**args)

            # Add tool result to messages
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result,
                }
            )

In [ ]:
run_agent()

In [ ]:
print(
    "- **Capital of France:** Paris  \n- **4847847 × 2409224:** **11,679,549,340,728**  \n- **Weather in Tokyo:** **72°F and sunny**  \n\n**Python “today’s date”, then sqrt using that date**\n- Today’s date (ISO): **2026-01-10**\n\nIf we “plug that date back into python” in a numeric way, a standard approach is to convert it to an **ordinal day number** (what Python’s `date.toordinal()` uses).  \nFor **2026-01-10**, the ordinal is **739626**, and:\n\n- **sqrt(739626) ≈ 859.899412**"
)